In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Input,Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as kb

import warnings
warnings.filterwarnings(action='ignore')

import autokeras as ak

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [2]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [3]:
def txtToDataframe(filename, flag1, flag2):
    file = open(filename, 'r')
    lines = file.readlines()
    datas = []
    for line in lines:
        txt = line.replace('   ', ' ').lstrip().rstrip().replace(' ', ',')
        data = txt.split(',')
        datas.append(data)
    df = pd.DataFrame(datas)
    df.columns = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'F8', 'AF4']
    df['label1']=flag1
    df['label2']=flag2
    return df


def getData(src) :
    file_list = os.listdir(src)
    
    rating = pd.DataFrame(pd.read_csv(src+'ratings.txt'))
    file_list.remove('ratings.txt')
    
    dataList=[]
    highList=[]
    lowList=[]
    
    print(rating.columns)
    j=0
    for i in rating['subject']:
        if i<10:
            num = str(0)+str(i)
        else:
            num = str(i)
        dataList.append(txtToDataframe(src+'sub'+num+'_hi.txt', 1,rating['test'][j]))
        dataList.append(txtToDataframe(src+'sub'+num+'_lo.txt', 0,rating['rest'][j]))
        highList.append(txtToDataframe(src+'sub'+num+'_hi.txt', 1,rating['test'][j]))
        lowList.append(txtToDataframe(src+'sub'+num+'_lo.txt', 0,rating['rest'][j]))
        j+=1
    return dataList, highList, lowList

src = './STEW Dataset/'
originalData, highData, lowData = getData(src)

Index(['subject', 'rest', 'test'], dtype='object')


In [4]:
mergedData = pd.concat([originalData[0],originalData[1]],ignore_index=True)
for i in range(2,len(originalData)):
    mergedData = pd.concat([mergedData,originalData[i]],ignore_index=True)
mergedData = mergedData.apply(pd.to_numeric)
mergedData

,AF3,F7,F3,FC5,T7,O1,O2,P8,T8,FC6,F4,F8,F8,AF4,label1,label2
0,4584.62,3902.05,4571.79,4589.23,4124.62,3825.13,4152.82,4579.49,4690.77,4260.00,4027.18,4385.13,4480.51,4230.77,1,8
1,4584.10,3895.90,4574.87,4567.69,4124.10,3827.18,4157.95,4585.13,4695.38,4268.21,4034.36,4380.00,4501.54,4197.44,1,8
2,4574.36,3893.85,4576.92,4572.82,4123.59,3829.23,4165.13,4590.26,4702.56,4281.54,4030.77,4366.67,4521.03,4176.41,1,8
3,4573.85,3906.15,4572.82,4612.31,4137.95,3830.77,4167.18,4596.92,4706.15,4285.64,4038.46,4376.41,4518.97,4207.18,1,8
4,4583.59,3911.28,4570.26,4621.03,4150.77,3833.85,4166.15,4597.44,4705.13,4282.05,4051.79,4387.18,4520.51,4220.00,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727995,4168.21,4124.62,4123.08,4181.54,4621.03,4051.79,4295.38,4333.85,4434.87,4038.97,4557.44,4714.87,4711.79,4698.46,0,1
1727996,4166.15,4123.08,4124.10,4180.51,4625.13,4053.85,4294.87,4326.15,4429.23,4035.90,4556.92,4716.92,4706.67,4693.85,0,1
1727997,4167.69,4120.51,4122.05,4178.97,4622.05,4051.79,4289.74,4311.28,4420.00,4024.62,4553.33,4712.82,4699.49,4688.72,0,1
1727998,4170.26,4120.00,4121.54,4181.03,4616.92,4050.77,4288.21,4312.31,4421.54,4024.10,4553.85,4712.31,4703.08,4693.85,0,1


In [5]:
label=mergedData['label1']
label2=mergedData['label2']
data=mergedData.drop(['label1','label2'],axis=1)

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)
data = pd.DataFrame(scaled, columns = data.columns, index=data.index)
data

,AF3,F7,F3,FC5,T7,O1,O2,P8,T8,FC6,F4,F8,F8,AF4
0,0.558932,0.452258,0.539627,0.553295,0.479273,0.449631,0.482450,0.542340,0.558323,0.505813,0.479338,0.521798,0.533296,0.494656
1,0.558867,0.451482,0.540022,0.550698,0.479207,0.449895,0.483076,0.543015,0.558871,0.506793,0.480193,0.521187,0.535799,0.490618
2,0.557644,0.451223,0.540286,0.551317,0.479142,0.450158,0.483952,0.543629,0.559726,0.508383,0.479766,0.519600,0.538119,0.488070
3,0.557580,0.452775,0.539759,0.556078,0.480956,0.450357,0.484202,0.544427,0.560153,0.508873,0.480681,0.520759,0.537874,0.491798
4,0.558803,0.453422,0.539430,0.557129,0.482576,0.450753,0.484076,0.544489,0.560032,0.508444,0.482268,0.522042,0.538057,0.493351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727995,0.506663,0.480335,0.481982,0.504143,0.541975,0.478809,0.499843,0.512926,0.527864,0.479439,0.542453,0.561057,0.560825,0.551323
1727996,0.506404,0.480140,0.482113,0.504018,0.542493,0.479074,0.499781,0.512004,0.527193,0.479073,0.542391,0.561301,0.560215,0.550765
1727997,0.506598,0.479816,0.481850,0.503833,0.542104,0.478809,0.499155,0.510224,0.526094,0.477727,0.541964,0.560813,0.559361,0.550143
1727998,0.506920,0.479752,0.481785,0.504081,0.541456,0.478677,0.498968,0.510347,0.526277,0.477665,0.542026,0.560752,0.559788,0.550765


In [7]:
def windowing_dataset(data, label, window_size):
    data_list = []
    label_list = []
    
    for i in range(0,len(data)//window_size,window_size):
        data_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i]))
    return np.array(data_list), np.array(label_list)

dataList, labelList = windowing_dataset(data,label,10)
#labelList = to_categorical(labelList,9)
print(data.shape, label.shape, dataList.shape, labelList.shape)

(1728000, 14) (1728000,) (17280, 10, 14) (17280,)


Autokeras

In [9]:
import autokeras as ak

In [10]:
from tensorflow.keras.utils import plot_model

In [11]:
from datetime import datetime

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(dataList,labelList, train_size=0.7, random_state=True ,stratify = labelList)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(12096, 10, 14) (5184, 10, 14) (12096, 9) (5184, 9)


In [13]:
x_train, x_test, y_train, y_test = train_test_split(data,label, train_size=0.7, random_state=True ,stratify = label)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1209600, 14) (518400, 14) (1209600,) (518400,)


In [14]:
trials=[20] #3,5,10, 
for trial in trials:
    clf_ = ak.ImageClassifier(overwrite=True, max_trials=trial)
    clf_.fit(x=X_train, y=Y_train, epochs=20)

    predicted_y = clf_.predict(X_test)
    print(predicted_y)
    loss, acc = clf_.evaluate(X_test, Y_test)
    print(clf_.evaluate(X_test, Y_test))
    print('Loss: %.3f   Accuracy: %.3f' % (loss,acc))

    model = clf_.export_model()
    model.summary()
    plot_model(model, show_shapes=True)
    tmp = int(acc*100)
    print(tmp)
    model.save('model/WindowingLabel2AutoKeras_'+
                str(datetime.now().strftime('%Y-%m-%d %H-%M-%S'))+' ACC_'+str(tmp)+'try.h5')

Trial 20 Complete [00h 40m 07s]
val_loss: 0.062281906604766846

Best val_loss So Far: 0.03783227875828743
Total elapsed time: 08h 47m 03s
Epoch 1/20
378/378 [==============================] - 155s 396ms/step - loss: 0.6295 - accuracy: 0.7794
Epoch 2/20
378/378 [==============================] - 150s 397ms/step - loss: 0.5640 - accuracy: 0.8123
Epoch 3/20
378/378 [==============================] - 150s 396ms/step - loss: 0.3796 - accuracy: 0.8676
Epoch 4/20
378/378 [==============================] - 149s 395ms/step - loss: 0.2578 - accuracy: 0.9096
Epoch 5/20
378/378 [==============================] - 150s 396ms/step - loss: 0.2018 - accuracy: 0.9334
Epoch 6/20
378/378 [==============================] - 149s 395ms/step - loss: 0.1500 - accuracy: 0.9460
Epoch 7/20
378/378 [==============================] - 150s 396ms/step - loss: 0.1263 - accuracy: 0.9569
Epoch 8/20
378/378 [==============================] - 150s 397ms/step - loss: 0.0871 - accuracy: 0.9683
Epoch 9/20
378/378 [==========

Model1 + (Model2-1 + Model2-2)

In [17]:
def windowing_dataset(data, label, label2, window_size):
    data_list = []
    label_list = []
    label2_list = []
    
    for i in range(0,len(data)//window_size,window_size):
        data_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i]))
        label2_list.append(np.array(label2.iloc[i]))
    return np.array(data_list), np.array(label_list), np.array(label2_list)

dataList, labelList, label2List = windowing_dataset(data,label,label2,10)
label2List = to_categorical(label2List,9)
print(data.shape, label.shape, dataList.shape, labelList.shape, label2List.shape)

(1728000, 14) (1728000,) (17280, 10, 14) (17280,) (17280, 9)


In [18]:
modelList=os.listdir('model/')
print(modelList)

['WindowingAutoKeras_2021-09-10 19-22-57 ACC_96try.h5', 'WindowingAutoKeras_2021-09-11 02-37-32 ACC_98try.h5', 'WindowingAutoKeras_97.PNG', 'WindowingLabel2AutoKeras_2021-09-16 05-31-15 ACC_37try.h5', 'WindowingLabel2AutoKeras_2021-09-16 08-07-53 ACC_95try.h5', 'WindowingLabel2AutoKeras_2021-09-16 10-29-28 ACC_96try.h5', 'WindowingLabel2AutoKeras_2021-09-16 23-21-15 ACC_99try.h5']


In [25]:
def makeInputShape(data):
    d=[]
    d.append(dataList[i])
    d=np.array(d)
    return d


In [28]:
model = tf.keras.models.load_model('model/WindowingAutoKeras_2021-09-11 02-37-32 ACC_98try.h5')

data2_1=[]
data2_2=[]
label2_1=[]
label2_2=[]

cnt = 0
for i in range(len(dataList)):
    pred=model.predict(makeInputShape(dataList[i]))
    if pred < 0.5: #rest
        data2_1.append(dataList[i])
        label2_1.append(label2List[i])
        if labelList[i] != 0 :
            #print('No: '+str(i)+', Count : ' + str(cnt)+', True : ' + str(labelList[i]) + ', Predict : ' + str(pred))
            cnt+=1
    else: #test
        data2_2.append(dataList[i])
        label2_2.append(label2List[i])
        if labelList[i] != 1 :
            #print('No: '+str(i)+', Count : ' + str(cnt) + ', True : ' + str(labelList[i]) + ', Predict : ' + str(pred))
            cnt+=1
            
print("Loss: "+str(cnt/len(dataList)))

data2_1 = np.array(data2_1)
data2_2 = np.array(data2_2)
label2_1 = np.array(label2_1)
label2_2 = np.array(label2_2)

No: 389, True : 1, Predict : [[0.44617373]]
No: 392, True : 1, Predict : [[0.3546907]]
No: 393, True : 1, Predict : [[0.11548146]]
No: 858, True : 1, Predict : [[0.02015549]]
No: 988, True : 1, Predict : [[0.15431085]]
No: 1278, True : 1, Predict : [[0.2706099]]
No: 1279, True : 1, Predict : [[0.17944951]]
No: 1280, True : 1, Predict : [[0.02453198]]
No: 1283, True : 1, Predict : [[0.45614785]]
No: 1558, True : 1, Predict : [[0.10921626]]
No: 1561, True : 1, Predict : [[0.02908372]]
No: 1673, True : 1, Predict : [[0.00560712]]
No: 1674, True : 1, Predict : [[0.12225463]]
No: 1825, True : 1, Predict : [[0.46229944]]
No: 1909, True : 1, Predict : [[0.20284943]]
No: 1950, True : 0, Predict : [[0.99801993]]
No: 2032, True : 0, Predict : [[0.8930559]]
No: 2103, True : 0, Predict : [[0.9540061]]
No: 2165, True : 0, Predict : [[0.99460536]]
No: 2232, True : 0, Predict : [[0.9218229]]
No: 2239, True : 0, Predict : [[0.67779624]]
No: 2246, True : 0, Predict : [[0.96905106]]
No: 2267, True : 0, 

No: 11463, True : 0, Predict : [[0.8421667]]
No: 11464, True : 0, Predict : [[0.8521352]]
No: 11465, True : 0, Predict : [[0.7710034]]
No: 11466, True : 0, Predict : [[0.59705174]]
No: 11467, True : 0, Predict : [[0.67481375]]
No: 11472, True : 0, Predict : [[0.72696716]]
No: 11473, True : 0, Predict : [[0.8104796]]
No: 11474, True : 0, Predict : [[0.620754]]
No: 11479, True : 0, Predict : [[0.80008036]]
No: 11480, True : 0, Predict : [[0.87485904]]
No: 11481, True : 0, Predict : [[0.8602879]]
No: 11482, True : 0, Predict : [[0.7233658]]
No: 11483, True : 0, Predict : [[0.8548598]]
No: 11484, True : 0, Predict : [[0.80953664]]
No: 11485, True : 0, Predict : [[0.7990977]]
No: 11487, True : 0, Predict : [[0.7243955]]
No: 11488, True : 0, Predict : [[0.7797243]]
No: 11490, True : 0, Predict : [[0.8623908]]
No: 11491, True : 0, Predict : [[0.8528979]]
No: 11492, True : 0, Predict : [[0.8649097]]
No: 11493, True : 0, Predict : [[0.8812827]]
No: 11494, True : 0, Predict : [[0.5284652]]
No: 1

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(data2_1,label2_1, train_size=0.7, random_state=True ,stratify = label2_1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

trials=[3,5,10,20]
for trial in trials:
    clf_ = ak.ImageClassifier(overwrite=True, max_trials=trial)
    clf_.fit(x=X_train, y=Y_train, epochs=20)

    predicted_y = clf_.predict(X_test)
    print(predicted_y)
    loss, acc = clf_.evaluate(X_test, Y_test)
    print(clf_.evaluate(X_test, Y_test))
    print('Loss: %.3f   Accuracy: %.3f' % (loss,acc))

    model = clf_.export_model()
    model.summary()
    plot_model(model, show_shapes=True)
    tmp = int(acc*100)
    print(tmp)
    model.save('model/model2_1AutoKeras_'+
                str(datetime.now().strftime('%Y-%m-%d %H-%M-%S'))+' ACC_'+str(tmp)+'try.h5')

Trial 20 Complete [01h 31m 19s]
val_loss: 0.07356321066617966

Best val_loss So Far: 0.05210596323013306
Total elapsed time: 00h 20m 13s
Epoch 1/20
Not enough memory, reduce batch size to 16.
Epoch 1/20
      1/Unknown - 1s 759ms/step - loss: 2.3254 - accuracy: 0.1250Not enough memory, reduce batch size to 8.
Epoch 1/20
664/664 [==============================] - 314s 474ms/step - loss: 0.5272 - accuracy: 0.8854
Epoch 2/20
664/664 [==============================] - 315s 474ms/step - loss: 0.1327 - accuracy: 0.9731
Epoch 3/20
664/664 [==============================] - 314s 473ms/step - loss: 0.0983 - accuracy: 0.9798
Epoch 4/20
664/664 [==============================] - 315s 474ms/step - loss: 0.0834 - accuracy: 0.9815
Epoch 5/20
664/664 [==============================] - 315s 474ms/step - loss: 0.0742 - accuracy: 0.9838
Epoch 6/20
664/664 [==============================] - 314s 474ms/step - loss: 0.0683 - accuracy: 0.9849
Epoch 7/20
664/664 [==============================] - 315s 474ms/

72/72 [==============================] - 17s 242ms/step
[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


72/72 [==============================] - 23s 260ms/step - loss: 0.2301 - accuracy: 0.9196


72/72 [==============================] - 23s 261ms/step - loss: 0.2301 - accuracy: 0.9196
[0.230147585272789, 0.919560432434082]
Loss: 0.230   Accuracy: 0.920


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 14)]     0                                            
__________________________________________________________________________________________________
cast_to_float32 (CastToFloat32) (None, 10, 14)       0           input_1[0][0]                    
__________________________________________________________________________________________________
expand_last_dim (ExpandLastDim) (None, 10, 14, 1)    0           cast_to_float32[0][0]            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 10, 14, 1)    3           expand_last_dim[0][0]            
______________________________________________________________________________________________

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(data2_2,label2_2, train_size=0.7, random_state=True ,stratify = label2_2)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

trials=[3,5,10,20]
for trial in trials:
    clf_ = ak.ImageClassifier(overwrite=True, max_trials=trial)
    clf_.fit(x=X_train, y=Y_train, epochs=20)

    predicted_y = clf_.predict(X_test)
    print(predicted_y)
    loss, acc = clf_.evaluate(X_test, Y_test)
    print(clf_.evaluate(X_test, Y_test))
    print('Loss: %.3f   Accuracy: %.3f' % (loss,acc))

    model = clf_.export_model()
    model.summary()
    plot_model(model, show_shapes=True)
    tmp = int(acc*100)
    print(tmp)
    model.save('model/model2_2AutoKeras_'+
                str(datetime.now().strftime('%Y-%m-%d %H-%M-%S'))+' ACC_'+str(tmp)+'try.h5')

Trial 20 Complete [02h 01m 10s]
val_loss: 0.1823974847793579

Best val_loss So Far: 0.10554419457912445
Total elapsed time: 03h 01m 00s
Epoch 1/20
Not enough memory, reduce batch size to 16.
Epoch 1/20
      1/Unknown - 1s 756ms/step - loss: 2.1602 - accuracy: 0.0625Not enough memory, reduce batch size to 8.
Epoch 1/20
849/849 [==============================] - 416s 490ms/step - loss: 0.7445 - accuracy: 0.7659
Epoch 2/20
849/849 [==============================] - 417s 491ms/step - loss: 0.3631 - accuracy: 0.8639
Epoch 3/20
849/849 [==============================] - 416s 490ms/step - loss: 0.3056 - accuracy: 0.8885
Epoch 4/20
849/849 [==============================] - 417s 491ms/step - loss: 0.2729 - accuracy: 0.9019
Epoch 5/20
849/849 [==============================] - 417s 491ms/step - loss: 0.2490 - accuracy: 0.9132
Epoch 6/20
849/849 [==============================] - 416s 490ms/step - loss: 0.2219 - accuracy: 0.9225
Epoch 7/20
849/849 [==============================] - 417s 491ms/s

91/91 [==============================] - 22s 242ms/step
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


91/91 [==============================] - 30s 272ms/step - loss: 0.4217 - accuracy: 0.8519


91/91 [==============================] - 30s 272ms/step - loss: 0.4217 - accuracy: 0.8519
[0.42173346877098083, 0.8518900275230408]
Loss: 0.422   Accuracy: 0.852


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 14)]     0                                            
__________________________________________________________________________________________________
cast_to_float32 (CastToFloat32) (None, 10, 14)       0           input_1[0][0]                    
__________________________________________________________________________________________________
expand_last_dim (ExpandLastDim) (None, 10, 14, 1)    0           cast_to_float32[0][0]            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 10, 14, 1)    3           expand_last_dim[0][0]            
______________________________________________________________________________________________

모델 shape확인

In [39]:
modelName=['model1','model2_1','model2_2']
modelList=[]
for name in modelName:
    modelList.append(tf.keras.models.load_model('model/'+name+'.h5'))
    
for i in range(len(modelList)):
    print(modelName[i])
    model = modelList[i]
    model.summary()

model1
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 14)]          0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 10, 14)            0         
_________________________________________________________________
expand_last_dim (ExpandLastD (None, 10, 14, 1)         0         
_________________________________________________________________
normalization (Normalization (None, 10, 14, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 12, 256)        2560      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 10, 256)        590080    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 5, 256)        

In [ ]:
modelName=['model1','model2_1','model2_2']
modelList=[]
for name in modelName:
    modelList.append(tf.keras.models.load_model('model/'+name+'.h5'))
    
for i in range(len(modelList)):
    print(modelName[i])
    model = modelList[i]
    model.summary()